In [1]:
from sklearn.datasets import make_blobs
import numpy as np
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

In [2]:
from sklearn.mixture import GaussianMixture

In [3]:
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10, random_state=42)

이 알고리즘이 추정한 파라미터를 확인해보겠습니다.

In [4]:
gm.weights_

array([0.39025715, 0.40007391, 0.20966893])

In [5]:
gm.means_

array([[ 0.05131611,  0.07521837],
       [-1.40763156,  1.42708225],
       [ 3.39893794,  1.05928897]])

In [6]:
gm.covariances_

array([[[ 0.68799922,  0.79606357],
        [ 0.79606357,  1.21236106]],

       [[ 0.63479409,  0.72970799],
        [ 0.72970799,  1.1610351 ]],

       [[ 1.14833585, -0.03256179],
        [-0.03256179,  0.95490931]]])

이 클래스는 기댓값-최대화(EM) 알고리즘을 사용하는데,<br>
이 알고리즘 또한 최적의 솔루션으로 수렴한다는 것이 보장되지 않습니다.

따라서 알고리즘을 수행할 횟수를 n\_init으로 설정해줄 필요가 있습니다.

그리고 알고리즘이 수렴했는지 여부와 반복 횟수를 확인할 수 있습니다.

In [7]:
gm.converged_

True

In [8]:
gm.n_iter_

4

수렴했고, 4번 반복했음을 알 수 있습니다.

이 모델은 새로운 샘플을 가장 비슷한 클러스터에 손쉽게 할당할 수 있습니다.(하드 군집)<br>
하드 군집을 위해서는 predict() 메소드를 사용합니다.

또는 특정 클러스터에 속할 확률을 예측할 수 있습니다.(소프트 군집)<br>
소프트 군집을 위해서는 predict_proba() 메소드를 사용합니다.

In [9]:
gm.predict(X)

array([0, 0, 1, ..., 2, 2, 2], dtype=int64)

In [10]:
gm.predict_proba(X)

array([[9.76741808e-01, 6.78581203e-07, 2.32575136e-02],
       [9.82832955e-01, 6.76173663e-04, 1.64908714e-02],
       [7.46494398e-05, 9.99923327e-01, 2.02398402e-06],
       ...,
       [4.26050456e-07, 2.15512941e-26, 9.99999574e-01],
       [5.04987704e-16, 1.48083217e-41, 1.00000000e+00],
       [2.24602826e-15, 8.11457779e-41, 1.00000000e+00]])

가우시안 혼합 모델은 생성 모델입니다.<br>
즉 이 모델에서 새로운 샘플을 만들 수 있다는 것입니다.<br>
(새로 만든 샘플은 클러스터 인덱스 순으로 정렬되어 있습니다.)

In [11]:
X_new, y_new = gm.sample(6)

In [12]:
X_new

array([[-0.86944074, -0.32767626],
       [ 0.29836051,  0.28297011],
       [-2.8014927 , -0.09047309],
       [ 3.98203732,  1.49951491],
       [ 3.81677148,  0.53095244],
       [ 2.84104923, -0.73858639]])

In [13]:
y_new

array([0, 0, 1, 2, 2, 2])

또한 주어진 위치에서 모델의 밀도를 추정할 수 있습니다.<br>
이를 위해 score_samples() 메소드를 사용합니다.

샘플이 주어지면 이 메소드는 그 위치의 확률 밀도 함수의 로그를 예측합니다.<br>
점수가 높을수록 밀도가 높습니다.

In [14]:
gm.score_samples(X)

array([-2.60768954, -3.57110232, -3.32987086, ..., -3.51347241,
       -4.39798588, -3.80746532])

이 점수의 지숫값을 계산하면 샘플의 위치에서 PDF 값을 얻을 수 있습니다.

이 값은 하나의 확률이 아니라 확률 밀도입니다.<br>
샘플이 특정 지역 안에 속할 확률을 예측하려면 그 지역에 대해 PDF를 적분해야 합니다.

특성이나 클러스터가 많거나 샘플이 적을 때는 EM이 최고의 솔루션으로 수렴하기 어렵습니다.<br>
이런 어려움을 줄이려면 학습할 파라미터 개수를 제한해야 합니다.

방법 중 하나는 클러스터의 모양과 방향의 범위를 제한하는 것입니다.
사이킷런에서는 covariance_type 매개변수에 다음 중 하나를 설정하면 됩니다.
- 'spherical'
- 'diag'
- 'tied'